In [1]:
import numpy as np

Here we see that if we group together we get  different amount of elements per group:

In [4]:
nFluExpForI=np.fromfile("/home/jkipen/ProtInfGPU/dev/OptKernel/data/nFluExpForI.bin", dtype=np.uint32)
nProt=len(nFluExpForI);
list_n_proteins_per_block=np.arange(150,200);
for n_proteins_per_block in list_n_proteins_per_block:
    n_chunks = int(np.ceil(len(nFluExpForI) / n_proteins_per_block))  
    list_sub_groups=np.array_split(nFluExpForI, n_chunks)
    max_value=np.max([np.sum(i) for i in list_sub_groups])
    print(f"n_proteins_per_block = {n_proteins_per_block}, max sum = {max_value}")

n_proteins_per_block = 150, max sum = 4979
n_proteins_per_block = 151, max sum = 4916
n_proteins_per_block = 152, max sum = 4929
n_proteins_per_block = 153, max sum = 5116
n_proteins_per_block = 154, max sum = 5116
n_proteins_per_block = 155, max sum = 5200
n_proteins_per_block = 156, max sum = 5152
n_proteins_per_block = 157, max sum = 5221
n_proteins_per_block = 158, max sum = 5186
n_proteins_per_block = 159, max sum = 5184
n_proteins_per_block = 160, max sum = 5184
n_proteins_per_block = 161, max sum = 5241
n_proteins_per_block = 162, max sum = 5237
n_proteins_per_block = 163, max sum = 5161
n_proteins_per_block = 164, max sum = 5274
n_proteins_per_block = 165, max sum = 5274
n_proteins_per_block = 166, max sum = 5545
n_proteins_per_block = 167, max sum = 5452
n_proteins_per_block = 168, max sum = 5442
n_proteins_per_block = 169, max sum = 5442
n_proteins_per_block = 170, max sum = 5583
n_proteins_per_block = 171, max sum = 5625
n_proteins_per_block = 172, max sum = 5625
n_proteins_

In [21]:
def loadSubMat(path,nProt,nSubRowsToLoad= (-1) ):
    if nSubRowsToLoad == -1:
        retVal=np.fromfile(path, dtype=np.float32);
    else:
        retVal=np.fromfile(path, dtype=np.float32,count=nProt*nSubRowsToLoad);
    retVal=retVal.reshape(-1, nProt)
    return retVal
def checkMatrixSanity(mat):
    if np.isnan(np.sum(mat)):
        print("There is a NaN!");
    if np.sum(np.sum(mat,axis=1)==0) > 0:
        print("There is a row that is zero!");

In [174]:
trueSubMat=loadSubMat("/home/jkipen/raid_storage/ProtInfGPU/data/PXgIrelTrue.bin",nProt,nSubRowsToLoad= 1000)
estSubMat=loadSubMat("/home/jkipen/raid_storage/ProtInfGPU/data/PXgIrelToCheckMemTransf.bin",nProt,nSubRowsToLoad= 1000)
print(np.sum(trueSubMat))
print(np.sum(estSubMat))
print(np.sum(np.sum(estSubMat,axis=1)==0))

98.92179
98.92179
0


In [145]:
print(estSubMat)
print(trueSubMat)

[[6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 [6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 [6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 ...
 [6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 [6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 [6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]]
[[6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 [6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 [6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 ...
 [6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 [6.566413e-08 6.566413e-08 6.566413e-08 ... 6.566413e-08 6.566413e-08
  6.566413e-08]
 [6.566413e-08 6.566413e-08 6.56

In [141]:
trueSubMat=loadSubMat("/home/jkipen/raid_storage/ProtInfGPU/data/PXgIrelTrue.bin",nProt)


KeyboardInterrupt: 

In [24]:
checkMatrixSanity(trueSubMat)

In [25]:
np.sum(trueSubMat)

51298.164

In [175]:
estSubMat=loadSubMat("/home/jkipen/raid_storage/ProtInfGPU/data/PXgIrelToCheck.bin",nProt)

In [143]:
checkMatrixSanity(estSubMat)

In [176]:
np.sum(estSubMat)

51298.156

Now lets compare the matrixes element-wise:

In [138]:
def compMats(matA,matB):
    den=np.maximum(matA,matB)
    den[den==0]=1e-6;
    num=np.abs(matA-matB)
    rel_dif=np.divide(num,den)
    print("The maximum relative difference was " + str(np.max(rel_dif)))

In [85]:
compMats(estSubMat,trueSubMat)

The maximum relative difference was 1.0


In [80]:
estSubMat=estSubMat[-50000:,:];

In [82]:
trueSubMat=trueSubMat[-50000:,:];

In [83]:
compMats(estSubMat,trueSubMat)

The maximum relative difference was 1.19206874e-07


In [149]:
den=np.maximum(trueSubMat,estSubMat)
num=np.abs(trueSubMat-estSubMat)
rel_dif=np.divide(num,den)
argmaxi=np.argmax(rel_dif)
print(argmaxi)
print(trueSubMat[int(argmaxi/nProt),int(argmaxi%nProt)])
print(estSubMat[int(argmaxi/nProt),int(argmaxi%nProt)])

20161324
0.067087054
0.13417405


In [173]:
print(np.argpartition(rel_dif.flatten(), -10)[-10:])
a=np.argpartition(rel_dif.flatten(), -10)[-10:];
rel_dif.flatten()[a]

[ 6880666  6880665  6880664  6880662  6880668 16122174  6880667  6880663
 10083070 20161324]


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.49999878, 0.        , 0.        , 0.4999991 , 0.4999998 ],
      dtype=float32)

In [171]:
argmaxi=16122174;
print(trueSubMat[int(argmaxi/nProt),int(argmaxi%nProt)])
print(estSubMat[int(argmaxi/nProt),int(argmaxi%nProt)])

0.013812063
0.02762406


In [148]:
compMats(estSubMat,trueSubMat)

The maximum relative difference was 0.4999998


In [166]:
estSubMat[976,(estSubMat[976,:]!=estSubMat[0,0])]

array([0.13417405], dtype=float32)

In [164]:
trueSubMat[976,(trueSubMat[976,:]!=trueSubMat[0,0])]

array([0.06708705], dtype=float32)

In [167]:
20161324%20642

14732

In [169]:
10083070%20642

9774

In [170]:
10083070/20642

488.4735006297839

In [172]:
16122174%20642

772